In [109]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

from sklearn.metrics import roc_auc_score

### Подготовка данных

##### Получение данных для анализа

In [110]:
PATH = './data/offers/data_part.json'

dataRead = []
with open(PATH, 'r', encoding='utf-8') as jsonfile:    
    dataRead = json.load(jsonfile)
    
data = pd.DataFrame(data=dataRead)
data.head()

,Unknown,Skilled,Advertising,Clip,Videoshooting,Model,TVShow,Photoshooting,Text
0,1,0,0,0,0,0,0,0,мужчин актер съемок клип съемк клип ищ геро му...
1,1,0,0,0,0,0,0,0,вниман москв т взлет ориентировочн девушк лет ...
2,1,0,0,0,0,0,0,0,х ф данил козловск пада аист мужчин лет рост р...
3,1,0,0,0,0,0,0,0,нов шанс попаст кастинг супер телевикторин уга...
4,1,0,0,0,0,0,0,0,кастинг июл девушек размер одежд парн работ ию...


##### Получение целевого признака data_y

In [111]:
columns = 'TVShow'
data_y = data[columns]
data_y.value_counts()

0    96
1     4
Name: TVShow, dtype: int64

##### Получение данных для обучения data_x

In [112]:
data_x = data['Text']
data_x.shape

(100,)

##### Разделение данных на train_x,train_y и test_x,test_y

In [113]:
TRAIN_PERCENT = 70
delimiter = int(len(data_y) * TRAIN_PERCENT / 100)

train_x = data_x[:delimiter]
test_x = data_x[delimiter:]
train_y = data_y[:delimiter]
test_y = data_y[delimiter:]

### CountVectorizer - преобразование текста

In [104]:
model_vectorizer = CountVectorizer()
vectorizer = model_vectorizer.fit(train_x)
feature_names = model_vectorizer.get_feature_names()

### TfidfVectorizer - преобразование текста

In [105]:
model_vectorizer = TfidfVectorizer()
vectorizer = model_vectorizer.fit(train_x)
feature_names = model_vectorizer.get_feature_names()

### Logistic regression

In [94]:
train_x = vectorizer.transform(train_x)
train_y = train_y

test_x = vectorizer.transform(test_x)
test_y = test_y

##### Обучим логистическую регрессию.

In [95]:
%%time
logit = LogisticRegression(n_jobs=-1, random_state=17)
logit.fit(train_x, train_y)

CPU times: user 4.73 ms, sys: 5.07 ms, total: 9.79 ms
Wall time: 16 ms


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=-1, penalty='l2', random_state=17,
                   solver='lbfgs', tol=0.0001, verbose=0, warm_start=False)

##### Доли правильных ответов на обучающей и тестовой выборках

In [96]:
round(logit.score(train_x, train_y), 3), round(logit.score(test_x, test_y), 3),

(1.0, 1.0)

##### Качесвто модели (roc_auc_score)

In [97]:
valid_pred = logit.predict_proba(test_x)[:,1]
test_y[70] = 1 # Если в выборке все 0 или 1
roc_auc = roc_auc_score(test_y, valid_pred)
print('Качество модели: ' + str(roc_auc))

Качество модели: 0.5172413793103448
